In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### ***balancing and splitting into M traininng sets***




In [ ]:
train_df1, test_df1 = train_test_split(data, test_size=0.2, random_state = seed, stratify=data['labels'])
train_df1, valid_df1 = train_test_split(train_df1, test_size=0.2, random_state = seed,  stratify=train_df1['labels'])

train_df1 = train_df1.reset_index(drop=True)
valid_df1 = valid_df1.reset_index(drop=True)
test_df1 = test_df1.reset_index(drop=True)

In [ ]:
# Step i: Random Oversampling to balance the classes

# Separate majority and minority classes
majority_class = train_df1[train_df1['labels'] == 0]
minority_class = train_df1[train_df1['labels'] == 1]

# Randomly sample majority class to match the number of minority class samples
minority_oversampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state = seed)

# Combine minority class and downsampled majority class
train_df1 = pd.concat([majority_class, minority_oversampled])

In [ ]:
bootstrapped_df = resample(train_df1, replace=True, n_samples= 2 * len(train_df1) , random_state = seed)

In [ ]:
# Step iii: Randomize and split into M different balanced training sets

shuffled_df = bootstrapped_df.sample(frac=1, random_state = seed)  # 'frac=1' shuffles all rows

# Reset the index of the shuffled DataFrame
shuffled_df.reset_index(drop=True, inplace=True)

In [ ]:
#shuffled_df = pd.read_excel('/content/drive/My Drive/shuffled_df.xlsx')

In [ ]:
# Define the number of subsets 'm'
m = 17

# Calculate the number of rows in each subset
subset_size = len(shuffled_df) // m

# Create a list to store the subsets
subsets = []

# Split the DataFrame into 'm' equal-sized subsets
for i in range(m):
    start_idx = i * subset_size
    end_idx = (i + 1) * subset_size if i < m - 1 else None
    subset = shuffled_df.iloc[start_idx:end_idx]
    subsets.append(subset)

### ***Feature selection***

Apply the results of features' selection on all trainsets and test set.

In [ ]:
filtered_df = []

for frame in subsets:
  frame = frame [merged_list]
  filtered_df.append(frame)

In [ ]:
test_df = test_df1[merged_list]

### ***Creating M base classifiers***

In [ ]:
import joblib
eclf2 = joblib.load('/content/drive/My Drive/ensemble_model.joblib')

In [ ]:
i = 0

# Assuming 'subsets' is a list of DataFrames
for subset in filtered_df:


        # Split the DataFrame into features (X) and target (y)
        y = subset['labels']
        X = subset.drop('labels', axis = 1)

        eclf2.clfs[i].fit(X, y)

        i = i + 1

In [ ]:
y_test = test_df['labels']
X_test = test_df.drop('labels', axis = 1)

In [ ]:
preds = eclf2.predict(X_test)

cm = confusion_matrix(y_test , preds)
#ConfusionMatrixDisplay(cm).plot()

In [ ]:
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ccr = (specificity+ sensitivity)/2
ppv = (tp / (tp + fp))
npv = (tn / (tn + fn))

print('sen:{}'.format(sensitivity))
print('spe:{}'.format(specificity))
print('CCR:{}'.format(ccr))
print('ppv:{}'.format(ppv))
print('nen:{}'.format(npv))

sen:0.6308154077038519
spe:0.7730288917281457
CCR:0.7019221497159989
ppv:0.2556599310671082
nen:0.9442682374263707


In [ ]:
from sklearn.metrics import roc_auc_score

probas = eclf2.predict_proba(X_test)
prob1 = probas[: , 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, prob1)

print("AUC-ROC Score:", auc_score)

AUC-ROC Score: 0.7671312335336224


In [ ]:
#shuffled_df.to_excel('/content/drive/My Drive/shuffled_df.xlsx', index=False)

In [ ]:
print('pass')